In [25]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame

data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",header=None)
data.columns = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','salary-class']
occ = data['occupation'].value_counts()
wor = data['workclass'].value_counts()
capacity = wor.sum() #32561

In [26]:
# prediction
# data extraction：5 continuous features
# age, education-num, capital-gain, capital-loss, hours-per-week
age = data['age']
eduNum = data['education-num']
capGain = data['capital-gain']
capLoss = data['capital-loss']
hpw = data['hours-per-week']
salary = data['salary-class']
assert ' ?' not in age.value_counts().index  # no missing data
assert ' ?' not in eduNum.value_counts().index
assert ' ?' not in hpw.value_counts().index
assert ' ?' not in capGain.value_counts().index
assert ' ?' not in capLoss.value_counts().index
assert ' ?' not in salary.value_counts().index
age = age.tolist()
eduNum = eduNum.tolist()
capGain = capGain.tolist()
capLoss = capLoss.tolist()
hpw = hpw.tolist()
salary = salary.tolist()
X = []
Y = []
for i in range(capacity):
    X.append([age[i],eduNum[i],hpw[i]])
    Y.append(salary[i] == ' >50K')
    
# SVM Classification
import scipy.optimize
import random
from sklearn import svm

scale_half = int(capacity/2)
X_train = X[:scale_half]
Y_train = Y[:scale_half]
X_test = X[scale_half:]
Y_test = Y[scale_half:]

c = 0.1
clf = svm.SVC(C=c, kernel='linear')
clf.fit(X_train, Y_train)

train_predictions = clf.predict(X_train)
test_predictions = clf.predict(X_test)

train_correct = 0
for i in range(scale_half):
    if train_predictions[i] == Y_train[i]:
        train_correct = train_correct + 1
train_acc = train_correct/float(scale_half)
test_correct = 0
for i in range(scale_half):
    if test_predictions[i] == Y_test[i]:
        test_correct = test_correct + 1
test_acc = test_correct/float(scale_half)
print "C=",c,"Accuracy of the predictor on the train data is:",train_acc
print "C=",c,"Accuracy of the predictor on the test data is:",test_acc


C= 0.1 Accuracy of the predictor on the train data is: 0.760626535627
C= 0.1 Accuracy of the predictor on the test data is: 0.757800982801


In [27]:
# Logistic Regression
from math import exp
from math import log
import scipy.optimize
import random
import numpy

def inner(x,y):
    return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
    return 1.0 / (1 + exp(-x))

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
    loglikelihood = 0
    for i in range(len(X)):
        logit = inner(X[i], theta)
        loglikelihood -= log(1 + exp(-logit))
        if not(y[i]==True):
            loglikelihood -= logit
    for k in range(len(theta)):
        loglikelihood -= lam * theta[k]*theta[k]
    #print ("ll =", loglikelihood)
    return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
    dl = [0.0]*len(theta)
    for k in range(len(theta)): 
        for i in range(len(X)):
            logit = inner(X[i], theta)
            dl[k] += X[i][k]*(exp(-logit))/(1 + exp(-logit))
            if not(y[i]==True):
                dl[k] -= X[i][k]
        dl[k] -= 2*lam*theta[k]
  # Negate the return value since we're doing gradient *ascent*
    return numpy.array([-x for x in dl])  

X = []
Y = []
for i in range(capacity):
    X.append([age[i],eduNum[i],capGain[i],capLoss[i],hpw[i]])
    Y.append(salary[i] == ' >50K')

scale_half = int(capacity/2)
X_train = X[:scale_half]
Y_train = Y[:scale_half]
X_test = X[scale_half:]
Y_test = Y[scale_half:]
theta,l,info = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, args = (X_train, Y_train, 1.0))
print "Final log likelihood =", -l

YPre_train = []
for i in range(scale_half):
    x_theta = 0
    for j in range(len(theta)):
        x_theta += X_train[i][j]*theta[j]
    if x_theta > 0:
        YPre_train.append(1)
    else:
        YPre_train.append(0)
count = 0
for i in range(scale_half):
    if Y_train[i] == YPre_train[i]:
        count = count + 1
acc = count/float(scale_half)
print "Accuracy of predictor in train data = ", acc

YPre_test = []
for i in range(scale_half):
    x_theta = 0
    for j in range(len(theta)):
        x_theta += X_test[i][j]*theta[j]
    if x_theta > 0:
        YPre_test.append(1)
    else:
        YPre_test.append(0)
count = 0
for i in range(scale_half):
    if Y_test[i] == YPre_test[i]:
        count = count + 1
acc = count/float(scale_half)
print "Accuracy of predictor in test data = ", acc

Final log likelihood = -8613.946116166113
Accuracy of predictor in train data =  0.797235872236
Accuracy of predictor in test data =  0.797604422604


In [28]:
# biased P_error with lambda
for lam in [0,0.01,0.1,1,10,100,1000,10000,100000]:
    theta,l,info = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, args = (X_train, Y_train, lam))
    #print "Final log likelihood =", -l

    YPre_train = []
    for i in range(scale_half):
        x_theta = 0
        for j in range(len(theta)):
            x_theta += X_train[i][j]*theta[j]
        if x_theta > 0:
            YPre_train.append(1)
        else:
            YPre_train.append(0)
    count = 0
    for i in range(scale_half):
        if Y_train[i] == YPre_train[i]:
            count = count + 1
    acc = count/float(scale_half)
    print "lambda = ", lam, "Accuracy of predictor in train data = ", acc

    YPre_test = []
    for i in range(scale_half):
        x_theta = 0
        for j in range(len(theta)):
            x_theta += X_test[i][j]*theta[j]
        if x_theta > 0:
            YPre_test.append(1)
        else:
            YPre_test.append(0)
    count = 0
    for i in range(scale_half):
        if Y_test[i] == YPre_test[i]:
            count = count + 1
    acc = count/float(scale_half)
    print "lambda = ", lam, "Accuracy of predictor in test data = ", acc

lambda =  0 Accuracy of predictor in train data =  0.797235872236
lambda =  0 Accuracy of predictor in test data =  0.797604422604
lambda =  0.01 Accuracy of predictor in train data =  0.797235872236
lambda =  0.01 Accuracy of predictor in test data =  0.797604422604
lambda =  0.1 Accuracy of predictor in train data =  0.797235872236
lambda =  0.1 Accuracy of predictor in test data =  0.797604422604
lambda =  1 Accuracy of predictor in train data =  0.797235872236
lambda =  1 Accuracy of predictor in test data =  0.797604422604
lambda =  10 Accuracy of predictor in train data =  0.797235872236
lambda =  10 Accuracy of predictor in test data =  0.797604422604
lambda =  100 Accuracy of predictor in train data =  0.797235872236
lambda =  100 Accuracy of predictor in test data =  0.797665847666
lambda =  1000 Accuracy of predictor in train data =  0.797174447174
lambda =  1000 Accuracy of predictor in test data =  0.79785012285
lambda =  10000 Accuracy of predictor in train data =  0.79742

In [29]:
def train(lam):
    theta,_,_ = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, pgtol = 10, args = (X_train, Y_train, lam))           
    return theta

def performance(theta, X, y):
    scores = [inner(theta,x) for x in X]
    predictions = [s > 0 for s in scores]
    correct = [(a==b) for (a,b) in zip(predictions,y)]
    acc = sum(correct) * 1.0 / len(correct)
    return acc

#BER Calculation
def ber(theta, X, y):
    scores = [inner(theta, x) for x in X]
    predictions = [s > 0 for s in scores]
    TP = [(a==1 and b==1) for (a,b) in zip(predictions, y)]
    TN = [(a==0 and b==0) for (a,b) in zip(predictions, y)]
    FP = [(a==1 and b==0) for (a,b) in zip(predictions, y)]
    FN = [(a==0 and b==1) for (a,b) in zip(predictions, y)]
    tp = sum(TP)
    tn = sum(TN)
    fp = sum(FP)
    fn = sum(FN)
    print("TP = "+str(tp)+" TN = "+str(tn)+" FP = "+str(fp)+" FN = "+str(fn))
    BER = 1-0.5*(float(fn)/(tp+fn)+float(tn)/(tn+fp))
    return BER

lam = 1.0

Theta_1 = train(lam)
acc1 = performance(Theta_1, X_train, Y_train)
acc2 = performance(Theta_1, X_test, Y_test)

print ("lambda = " + str(lam) + ":\taccuracy of training set = " + str(acc1))
print ("lambda = " + str(lam) + ":\taccuracy of test set = " + str(acc2))

BER_1 = ber(Theta_1, X_test, Y_test)
print("BER of classifier on test set is "+str(BER_1))

lambda = 1.0:	accuracy of training set = 0.7972358722358722
lambda = 1.0:	accuracy of test set = 0.797616854001597
TP = 1051 TN = 11935 FP = 402 FN = 2893
BER of classifier on test set is 0.149532818706
